In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, timedelta

/Users/jenifervivar/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
stocks = ['AAPL', 'MSFT', 'TSLA', 'NFLX', 'GE', 'AUY', 'PLUG', "SPY", "IWM", "DIA"]
tickers = ['AAPL', 'MSFT', 'TSLA', 'NFLX', 'GE', 'AUY', 'PLUG']
benches = ["SPY", "IWM", "DIA"]
data = yf.download(stocks, start="2020-01-01", end="2020-12-31")["Adj Close"] #ad_close
data

[*********************100%***********************]  10 of 10 completed


,AAPL,AUY,DIA,GE,IWM,MSFT,NFLX,PLUG,SPY,TSLA
Date,,,,,,,,,,
2020-01-02,73.785919,3.705322,275.016510,94.461693,161.172165,156.924377,329.809998,3.240000,312.395050,86.052002
2020-01-03,73.068565,3.676893,272.824982,94.778412,160.540222,154.970413,325.899994,3.230000,310.029572,88.601997
2020-01-06,73.650803,3.667416,273.453857,96.124481,160.754120,155.370987,335.829987,3.820000,311.212311,90.307999
2020-01-07,73.304405,3.724276,272.272217,95.411858,160.219391,153.954346,330.750000,3.810000,310.337219,93.811996
2020-01-08,74.483597,3.515791,273.854004,94.540863,160.715210,156.406586,339.260010,4.080000,311.991211,98.428001
...,...,...,...,...,...,...,...,...,...,...
2020-12-23,129.801270,5.468959,293.324799,86.425804,196.364899,218.222198,514.479980,36.139999,360.056091,645.979980
2020-12-24,130.802353,5.430648,294.045197,84.754585,196.236725,219.930298,513.969971,35.580002,361.456848,661.770020
2020-12-28,135.480576,5.392336,295.972961,84.675011,195.507019,222.112335,519.119995,33.570000,364.562042,663.690002


# Part 1

In [20]:
weight =[1,1,1,1,1,1,1]

## Volatility


In [63]:
returnn = data[tickers].pct_change(66)
volatility = returnn.std()
volatility_annual = (volatility*np.sqrt(252)).to_numpy()
volatility_annual

array([ 3.49940079,  1.84446464,  8.10818243,  1.98081315,  5.81164953,
        4.11938527, 10.37034696])

## total & annual return

Daily_pct_change = todays_price - yesterdays_price/yesterdays_price *100
annualized% = need to multiply our daily standard deviation by the square root of 252

In [46]:
data2 = yf.download(stocks, period ='10y')["Adj Close"]
annualized_pct= ((data2[tickers].pct_change(len(data2)-1).iloc[-1])*np.sqrt(252)).to_numpy()
total_return = (data2[tickers].pct_change(len(data2)-1).iloc[-1]).to_numpy()
print(annualized_pct,total_return)

[*********************100%***********************]  10 of 10 completed
[ 108.58055355  145.30039506 1759.09916676  239.36282513   -7.57582519
   -9.66012199  210.78586238] [  6.83993195   9.15306454 110.81283159  15.07844068  -0.47723213
  -0.60853049  13.27826123]


## Beta
Beta is a measure of a stock’s volatility in relation to the overall market (S&P 500). The S&P 500 index has Beta 1.
High-beta stocks are supposed to be riskier but provide higher potential return. While, low-beta stocks pose less risk but also lower returns.

**Beta above 1: stock is more volatile than the market, but expects higher return.**
**Beta below 1: stock with lower volatility, and expects less return.**

In [ ]:
log_returns = np.log(data/data.shift())
cov = log_returns.cov()
beta_SPY =[]
beta_IWM = []
beta_DIA = []
for stock in tickers:
    var = log_returns[stock].var()
    for bench in benches:
        beta = cov.loc[bench, stock]/var
        if bench == 'SPY':
            beta_SPY.append(beta)
        if bench == 'IWM':
            beta_IWM.append(beta)
        if bench =='DIA':
            beta_DIA.append(beta)

## Max Drawdown — 
The worst possible return one could see, if they had bought high and sold low.
Maximum drawdown measures the maximum fall in the value of the investment, as given by the difference between the value of the lowest trough and that of the highest peak before the trough.

In [7]:
def Max_drawdown(df, window=5, draw_type = 'max') :
    if draw_type =='avg':
         Roll_Max = df.rolling(window,min_periods=1).mean()
    else:
        Roll_Max = df.rolling(window,min_periods=1).max()
        
    Daily_Drawdown= df/Roll_Max -1

    Max_Daily_Drawdown = Daily_Drawdown.rolling(window, min_periods=1).min()
    
    return Max_Daily_Drawdown

In [8]:
max_drawdown = (Max_drawdown(data[tickers]).resample('Y').mean()).to_numpy()
max_drawdown

array([[-0.04105441, -0.03968375, -0.0813381 , -0.04553247, -0.07227611,
        -0.06962732, -0.08661327]])

In [9]:
avg_drawdown = (Max_drawdown(data[tickers], draw_type = 'avg').resample('Y').mean()).to_numpy()
avg_drawdown

array([[-0.01827056, -0.01752702, -0.03405567, -0.02041568, -0.0343981 ,
        -0.03321462, -0.03533574]])

Here we got the logarithmic return, or continuously compounded return. This return is the nominal return with the interest compounding every millisecond. To see how it is close to effective interest rate. Here we calculated the daily logarithmic return.

In [65]:
annualized_pct

array([ 108.58055355,  145.30039506, 1759.09916676,  239.36282513,
         -7.57582519,   -9.66012199,  210.78586238])

#annulaized return

Annualized Return=(1+Cumulative Return)^(365/Days Held)	 −1

In [66]:
vals = {'Tickers': tickers, "Weight": weight, "annual_volatility":volatility_annual, "beta_SPY":beta_SPY, "beta_IWM":beta_IWM,
       "beta_DIA": beta_DIA, "avg_draw":avg_drawdown[0], "max_draw":max_drawdown[0], "Total_Return": total_return,
        "Annualized_return":annualized_pct}
risk_table = pd.DataFrame(vals)
risk_table

,Tickers,Weight,annual_volatility,beta_SPY,beta_IWM,beta_DIA,avg_draw,max_draw,Total_Return,Annualized_return
0,AAPL,1,3.499401,0.600835,0.583039,0.613052,-0.018271,-0.041054,6.839932,108.580554
1,MSFT,1,1.844465,0.670543,0.656226,0.686987,-0.017527,-0.039684,9.153065,145.300395
2,TSLA,1,8.108182,0.185986,0.214848,0.187835,-0.034056,-0.081338,110.812832,1759.099167
3,NFLX,1,1.980813,0.359710,0.334870,0.336624,-0.020416,-0.045532,15.078441,239.362825
4,GE,1,5.811650,0.369178,0.486225,0.434747,-0.034398,-0.072276,-0.477232,-7.575825
5,AUY,1,4.119385,0.143555,0.146942,0.137052,-0.033215,-0.069627,-0.608530,-9.660122
6,PLUG,1,10.370347,0.175357,0.230744,0.192523,-0.035336,-0.086613,13.278261,210.785862


# part 2